In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

import os
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras import Model

#### Importing the model

In [2]:
vggish_model = hub.load('https://kaggle.com/models/google/vggish/frameworks/TensorFlow2/variations/vggish/versions/1')

In [3]:
def load_and_preprocess_audio(file_path, target_duration=30, target_sr=16000):
    # Load audio file with original sampling rate
    audio, sr = librosa.load(file_path, sr=None)
    
    # Resample if needed
    if sr != target_sr:
        audio = librosa.resample(y=audio, orig_sr=sr, target_sr=target_sr)
        sr = target_sr
    
    # Ensure target duration
    target_length = int(sr * target_duration)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    elif len(audio) > target_length:
        audio = audio[:target_length]
    
    
    # Run the model, check the output.
    embeddings = vggish_model(audio)
    embeddings.shape.assert_is_compatible_with([None, 128])
    return embeddings

In [4]:
dataset_path = "C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/enrollment"
speakers = os.listdir(dataset_path)

In [5]:
embeddings = []
labels = []

file_paths = []
for filename in speakers:
    for files in os.listdir(os.path.join(dataset_path, filename)):
        file_path = os.path.join(dataset_path, filename,files)
        embedding = load_and_preprocess_audio(file_path)
        embeddings.append(embedding)
        labels.append(filename)
print(embeddings)
print(labels)

[<tf.Tensor: shape=(31, 128), dtype=float32, numpy=
array([[ 0.06515312, -0.04226655,  0.09394749, ...,  0.5642941 ,
         0.19134592, -0.27165246],
       [ 0.11619298,  0.09190807,  0.0437244 , ...,  0.7108487 ,
         0.05969664, -0.33334   ],
       [-0.03707516, -0.35628417,  0.34492695, ..., -0.23619649,
         0.07432567, -0.3839072 ],
       ...,
       [-0.26968613, -0.05457264, -0.1370081 , ..., -0.16206095,
         0.22875692,  0.17773895],
       [-0.26968613, -0.05457264, -0.1370081 , ..., -0.16206095,
         0.22875692,  0.17773895],
       [-0.26968613, -0.05457264, -0.1370081 , ..., -0.16206095,
         0.22875692,  0.17773895]], dtype=float32)>, <tf.Tensor: shape=(31, 128), dtype=float32, numpy=
array([[ 0.10035817,  0.1862867 ,  0.06903632, ...,  0.80040526,
         0.12545386, -0.29154682],
       [ 0.07294646, -0.03365093, -0.13683122, ...,  0.8876507 ,
         0.21817365, -0.33590043],
       [-0.05734241, -0.07055569,  0.31567675, ...,  0.46167123,
  

In [6]:
len(embeddings)

225

In [7]:
embeddings[0].shape

TensorShape([31, 128])

In [8]:
#Now let's wirte denoised .wav files in a new folder (New folder I named as denoise_new )
denoised_folder = "C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/denoised_new"
denoised_labels = os.listdir(denoised_folder)

In [9]:
test_embeddings = []
test_labels = []

test_file_paths = []
for i, filename in enumerate(denoised_labels):
    file_path = os.path.join(denoised_folder, filename)
    embedding = load_and_preprocess_audio(file_path)
    test_embeddings.append(embedding)
    test_labels.append(filename[:-4])
    print(f"File {i} : {filename} embedding generated")
print(test_labels)

File 0 : 0000f88619.wav embedding generated
File 1 : 000f6d0072.wav embedding generated
File 2 : 001dd0cb47.wav embedding generated
File 3 : 001f7db31a.wav embedding generated
File 4 : 001f86a5bf.wav embedding generated
File 5 : 002724b7ea.wav embedding generated
File 6 : 0042734703.wav embedding generated
File 7 : 00465a38b0.wav embedding generated
File 8 : 004adba134.wav embedding generated
File 9 : 005ae9cb2b.wav embedding generated
File 10 : 006a19dfc1.wav embedding generated
File 11 : 006ce8656f.wav embedding generated
File 12 : 00806ad24b.wav embedding generated
File 13 : 0086c2506c.wav embedding generated
File 14 : 008a2d1b07.wav embedding generated
File 15 : 008a7764f2.wav embedding generated
File 16 : 0094b5dfc9.wav embedding generated
File 17 : 009ba16c1a.wav embedding generated
File 18 : 00a49d6b07.wav embedding generated
File 19 : 00a794c50e.wav embedding generated
File 20 : 00a931d755.wav embedding generated
File 21 : 00a99c3526.wav embedding generated
File 22 : 00b83bd14a

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
file_path = "C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/signle-channel-trials.trl"
file2 = open('output_with_new_denosing.txt','w')
with open(file_path, 'r') as file:
    for line in file:
        speaker = line.split()[0]
        sample = line.split()[1]
        for i, label in enumerate(labels):
            if label == speaker:
                speaker_index = i
                break  # Break the loop once a match is found
        for i, label in enumerate(test_labels):
            if label == sample:
                index = i
                break  # Break the loop once a match is found
        test_array = test_embeddings[index]
        speaker_array = embeddings[speaker_index]
        array1_2d = tf.reshape(test_array, (1, -1))
        array2_2d = tf.reshape(speaker_array, (1, -1))
        similarity = cosine_similarity(array1_2d, array2_2d)
        file2.write(f"{speaker}\t{sample}\t{similarity[0][0]}\n")
file2.close()

In [151]:
type(test_embeddings[0])

tensorflow.python.framework.ops.EagerTensor

In [152]:
array1 = test_embeddings[0]
array2 = embeddings[0]
array1_2d = tf.reshape(array1, (1, -1))
array2_2d = tf.reshape(array2, (1, -1))
similarity = cosine_similarity(array1_2d, array2_2d)


In [153]:
similarity

array([[0.5140271]], dtype=float32)

In [154]:
from sklearn.decomposition import FactorAnalysis

In [155]:
# Convert embeddings to NumPy arrays
embeddings_np = np.array([embedding.numpy() for embedding in embeddings])
# Reshape to (num_samples * num_features, num_dimensions)
embeddings_flattened = embeddings_np.reshape(embeddings_np.shape[0], -1)

# Apply Factor Analysis
fa = FactorAnalysis(n_components=1)
ivectors = fa.fit_transform(embeddings_flattened)

In [156]:
import librosa

def get_audio_duration(file_path):
    time = []
    for file in os.listdir(file_path):
    # Load the audio file
        path = os.path.join(file_path, file)
        audio, sr = librosa.load(path, sr=None)
    
    # Calculate the duration in seconds
        duration = librosa.get_duration(y=audio, sr=sr)
        time.append(duration)
    return time

# Example usage
file_path = dataset_path
duration = get_audio_duration(file_path)
print(max(duration))


48.0371875
